In [ ]:
#----------------------------------------
#  CSC 315 / 615 Spring 2023
#  Project 4 Animal Species Taxonomy
#
#  Nicey Raiyani
#----------------------------------------

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bisect

#------------------------------------------
# The TaxonomyNode class
#------------------------------------------

class TaxonomyNode:
    def __init__(self, name, category):
        #self.name must contain the value of the CSV table such as Animalia, Chortada, or Reptilia
        self.name = name
        #self.category must contain the csv column headers such as Kingdom, Phylum, or Class
        self.category = category
        #self.children must contain a list of TaxonomyNode children
        self.children = []

    def __lt__(self, other):
        return self.name < other.name

    def __eq__(self, other):
        return self.name == other.name
    
    #addChild must insert a new TaxonomyNode into the list self.children upon insertion, the elements of self.children must be sorted by name
    def addChild(self, name, category):
        node = TaxonomyNode(name, category)
        bisect.insort(self.children, node)

    #hasChild must return True if self.children contains a TaxonomyNode with the specified name and category hasChild must return False otherwise
    def hasChild(self, name, category):
        return TaxonomyNode(name, category) in self.children

    #This function must return the TaxonomyNode from self.children with the specified name,category If self.children does not have such a node, This function should return None
    def getChild(self, name, category):
        for child in self.children:
            if child.name == name and child.category == category:
                return child
        return None

#------------------------------------------
# The TaxonomyTree class
#------------------------------------------

class TaxonomyTree:
    def __init__(self):
        self.root = TaxonomyNode("", "")

    # The addSpecies function must build the hierarchy of TaxonomyNodes starting from self.root all the way down to sub-species. given a list of names and categories.
    def addSpecies(self, names, categories):
        current_node = self.root
        for name, category in zip(names, categories):
            if not current_node.hasChild(name, category):
                current_node.addChild(name, category)
            current_node = current_node.getChild(name, category)

    def print(self):
        TaxonomyTree.print_internal(self.root, 1, "", "")

    @staticmethod
    def print_internal(node, lineno, number_str, name_str):
        if node.category != "":
            current_name_str = f"{name_str}{node.name}" if name_str else node.name
            print(f"{lineno: <10}{number_str: <30}{current_name_str}.")
            lineno += 1

        for i, child in enumerate(node.children):
            child_name_str = f"{name_str}{node.name}"
            if node.category != "":
                child_name_str += "."
            lineno = TaxonomyTree.print_internal(
                child,
                lineno,
                f"{number_str}{i + 1}.",
                child_name_str,
            )

        return lineno

    def printScientificNames(self):
        names = TaxonomyTree.listScientificNames_internal(self.root)
        names.sort()
        for name in names:
            print(name)
            
    # This function returns a list of all scientific names of all the unique species
    @staticmethod
    def listScientificNames_internal(node):
        scientific_names = []

        if not node.children:
            if node.category == "Scientific.Name":
                scientific_names.append(node.name)

        for child in node.children:
            scientific_names.extend(TaxonomyTree.listScientificNames_internal(child))

        return scientific_names

    
#------------------------------------------
# Main code
#   Do not Modify
#------------------------------------------

# Read the Pandas dataframe
df = pd.read_csv("taxonomy.csv")
rows = df.shape[0]
cols = df.shape[1]
categories = list(df.columns)[1:cols]

# Construct a Taxonomy Tree
tree = TaxonomyTree()

# Insert species into the tree row by row
for r in range(rows):
    names = list(df.iloc[r, 1:cols])
    tree.addSpecies(names, categories)

# Print the contents of the TaxonomyTree
tree.print()

# Extra Credit: print all scientific names in alphabetical order
tree.printScientificNames()
